In [23]:
import numpy as np
import pandas as pd
import sys
import os
from sklearn.datasets import fetch_covtype
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

sys.path.append(os.path.abspath(".."))
from common import common

In [24]:
base_path = common.base_path

In [25]:
def get_mba_df():
    config = {
        'TARGET_COLUMN': 'admission',
        # List of numerical columns (these are to be standardized)
        'NUMERICAL_COLUMNS': ['gpa', 'gmat', 'work_exp'],
        # List of categorical columns (these are to be one hot encoded)
        'CATEGORICAL_COLUMNS': [],
        # List of ordinal columns (these are to be label encoded)
        'ORDINAL_COLUMNS': ['gender', 'international', 'major', 'race', 'work_industry'],
    }
    target_column = config['TARGET_COLUMN']
    all_df = pd.read_csv(f'{base_path}/datasources/mba/MBA.csv')
    
    # Headers of column
    main_labels = all_df.columns
    
    print('Normal class: ', all_df[target_column].mode())
    return (all_df, main_labels, config)

In [31]:
def get_processed_mba_df():
    all_df, main_labels, config = get_mba_df()
    # print('main_labels', main_labels)
    target_column = config['TARGET_COLUMN']

    # Preprocess
    all_df = all_df.drop('application_id',axis=1)
    all_df['race'] = all_df['race'].fillna('Unknown')
    all_df['admission'] = all_df['admission'].fillna('Deny')
    
    # Label Encoder
    le, all_df = common.label_encode(all_df, [target_column])
    
    config['TARGET_DICT'] = {index: label for index, label in enumerate(le.classes_)}
    config['INV_TARGET_DICT'] = {v: k for k, v in config['TARGET_DICT'].items()}
    print('TARGET_DICT', config['TARGET_DICT'])
    le, all_df = common.label_encode(all_df, config['ORDINAL_COLUMNS'])
    
    config['NORMAL_TARGET'] = config['INV_TARGET_DICT']['Deny']
    print('NORMAL_TARGET', config['NORMAL_TARGET'])

    # One Hot Encoder
    ohe, all_df = common.one_hot_encode(all_df, config['CATEGORICAL_COLUMNS'])
    
    main_labels = list(all_df.columns)
    print('main_labels', main_labels)
    
    return (all_df, main_labels, config)